# Wczytywanie danych

In [50]:
import torch
from torch_geometric.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Wczytaj zbiór uczący
train_dataset = torch.load('train_dataset.pt')
# wczytaj zbiór testowy
test_dataset = torch.load('test_dataset.pt')

print(len(train_dataset))
print(len(test_dataset))

# Definiuj DataLoader dla zbiorów uczącego i testowego
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

184
46


/home/franek/anaconda3/lib/python3.11/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


# ChGW0

In [51]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GraphConv

class GraphRegressionModel(nn.Module):
    def __init__(self, num_node_features=1, hidden_dim=64, output_dim=1, dropout=0.5):
        super(GraphRegressionModel, self).__init__()
        name = 'ChGW0'

        # Graph Convolutional Layer
        self.conv1 = GraphConv(num_node_features, hidden_dim)

        # Fully Connected Layers with Dropout
        self.fc1 = nn.Linear(hidden_dim + 2, hidden_dim)
        self.dropout = nn.Dropout(p=dropout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index, edge_weight, global_feature = data.x, data.edge_index, data.edge_weight, data.global_feature

        # Apply Graph Convolution
        x = self.conv1(x, edge_index, edge_weight=edge_weight)

        # Global features concatenation
        global_feature = global_feature.expand(x.size(0), -1)  # Dostosuj global_feature do rozmiaru x
        x = torch.cat([x, global_feature], dim=1)

        # Fully Connected Layers with Dropout
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x
    
    def loss(self, pred, score):
        return F.mse_loss(pred, score)

# Zbieranie danych

In [52]:
from hyperopt import fmin, tpe, hp

# Zdefiniuj funkcję oceny (score function)
def objective(params):
    hidden_dim = int(params['hidden_dim'])
    dropout = params['dropout']

    model = GraphRegressionModel(hidden_dim=hidden_dim, dropout=dropout).to(device)
    # Definiuj optymalizator, liczbę epok, itp.
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    num_epochs = 200
    
    for epoch in range(num_epochs):
        # Trening i ewaluacja modelu
        for data in train_loader:
            data.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = model.loss(output, data.y.view(-1, 1).float())
            loss.backward()
            optimizer.step()
    
    # Ewaluacja na zbiorze testowym
    test_loss = 0.0  
    for data in test_loader:
        data.to(device)
        output = model(data)
        test_loss += model.loss(output, data.y.view(-1, 1).float()).item()
    
    # Zwróć funkcję oceny (score)
    return test_loss

# Przestrzeń poszukiwań dla hyperopt
space = {
    'hidden_dim': hp.quniform('hidden_dim', 32, 256, 32),  # Przeszukuj wartości co 32
    'dropout': hp.uniform('dropout', 0.05, 0.8),
}

# Minimalizacja funkcji oceny za pomocą algorytmu TPE
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10)  # Dla przykładu ustawiono max_evals na 10
print("Najlepsze hiperparametry:", best)
# Open the file in 'a' mode (append mode)

import torch

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Ustawienie optymalnych hiperparametrów
optimal_hidden_dim = int(best['hidden_dim'])
optimal_dropout = best['dropout']

#optimal_hidden_dim = 192
#optimal_dropout = 0.06

# Inicjalizacja modelu z optymalnymi hiperparametrami
optimal_model = GraphRegressionModel(hidden_dim=optimal_hidden_dim, dropout=optimal_dropout).to(device)



# Definiuj optymalizator, liczbę epok, itp.
optimizer = torch.optim.Adam(optimal_model.parameters(), lr=0.001)
num_epochs = 100

# Listy do śledzenia train loss i test loss
train_losses = []
test_losses = []

# Trening modelu
for epoch in range(num_epochs):
    if epoch % 10 == 0:
        print("Epoch:", epoch)
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = optimal_model(data)
        loss = optimal_model.loss(output, data.y.view(-1, 1).float())
        loss.backward()
        optimizer.step()

sample_data = test_dataset[0].to(device)

# Przeprowadzenie predykcji na przykładowym obiekcie
optimal_model.eval()
with torch.no_grad():
    prediction = optimal_model(sample_data)[0][0].item()

# Wyświetlenie wyników
print("Przewidywana wartość:", prediction)
print("Rzeczywista wartość:", sample_data.y.item())

total_accuracy = 0.0

# Iteracja przez cały zbiór testowy
for i, sample_data in enumerate(test_dataset):
    # Przeprowadzenie predykcji na przykładowym obiekcie
    optimal_model.eval()
    with torch.no_grad():
        prediction = optimal_model(sample_data.to(device))[0][0].item()
    
    # Wyświetlenie wyników
    print(f"Przykład {i + 1}:")
    print("Przewidywana wartość:", prediction)
    print("Rzeczywista wartość:", sample_data.y.item())
    
    # Obliczenie procentowej jakości regresji
    actual_value = sample_data.y.item()
    accuracy = 100 * (1 - abs(prediction - actual_value) / actual_value)
    total_accuracy += accuracy
    
    # print(f"Procentowa jakość regresji: {accuracy:.2f}%\n")

# Obliczenie średniej jakości procentowej
average_accuracy = total_accuracy / len(test_dataset)
print(f"Średnia procentowa jakość regresji dla wszystkich przykładów: {average_accuracy:.2f}%")

from sklearn.metrics import r2_score, mean_squared_error

# Załóżmy, że y_true i y_pred to twoje rzeczywiste i przewidywane wartości.
#optimal_model.load_state_dict(torch.load("model_big"))

total_accuracy = 0.0
y_true = []
y_pred = []

# Iteracja przez cały zbiór testowy
for i, sample_data in enumerate(test_dataset):
    # Przeprowadzenie predykcji na przykładowym obiekcie
    optimal_model.eval()
    with torch.no_grad():
        prediction = optimal_model(sample_data)[0][0].item()
        y_pred.append(prediction)
    
    y_true.append(sample_data.y.item())
    
    # Wyświetlenie wyników
    # print(f"Przykład {i + 1}:")
    # print("Przewidywana wartość:", prediction)
    # print("Rzeczywista wartość:", sample_data.y.item())
    
    # Obliczenie procentowej jakości regresji
    actual_value = sample_data.y.item()
    accuracy = 100 * (1 - abs(prediction - actual_value) / actual_value)
    total_accuracy += accuracy
    
    # print(f"Procentowa jakość regresji: {accuracy:.2f}%\n")

# Obliczenie średniej jakości procentowej
average_accuracy = total_accuracy / len(test_dataset)
r2 = r2_score(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
rmse = mse**(1/2)

print(f"R2 Score = {r2:.4f}")
print(f"MSE Score = {mse:.4f}")
print(f"RMSE Score = {rmse:.4f}")

y_mean = sum(y_true) / len(y_true)
print(f"Jaki procent zmiennej zależnej stanowi błąd RMSE? {rmse / y_mean * 100:.4f}%")

with open("file.txt", "a") as file:
    # Append the text to the file
    file.write("\n" + "ChGW0")  # Add a new line before appending
    file.write("\n" + "R2 "+ str(r2) )
    file.write("\n" + "MSE "+ str(mse))
    file.write("\n" + "RMSE "+ str(rmse))
    file.write("\n" + "Awerage "+ str(average_accuracy))

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/tmp/ipykernel_4183/570194302.py:37: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([28, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(pred, score)



100%|██████████| 10/10 [09:37<00:00, 57.79s/trial, best loss: 123719.13114929199]
Najlepsze hiperparametry: {'dropout': 0.1229458687006038, 'hidden_dim': 128.0}
Epoch: 0
Epoch: 10
Epoch: 20
Epoch: 30
Epoch: 40
Epoch: 50
Epoch: 60
Epoch: 70
Epoch: 80
Epoch: 90
Przewidywana wartość: 630.7854614257812
Rzeczywista wartość: 874.5499877929688
Przykład 1:
Przewidywana wartość: 630.7854614257812
Rzeczywista wartość: 874.5499877929688
Przykład 2:
Przewidywana wartość: 576.5301513671875
Rzeczywista wartość: 569.8499755859375
Przykład 3:
Przewidywana wartość: 552.2318115234375
Rzeczywista wartość: 450.54998779296875
Przykład 4:
Przewidywana wartość: 640.8490600585938
Rzeczywista wartość: 839.4600219726562
Przykład 5:
Przewidywana wartość: 603.0614624023438
Rzeczywista wartość: 705.010009765625
Przykład 6:
Przewidywana wartość: 599.8145141601562
Rzeczywista wartość: 697.9400024414062
Przykład 7:
Przewidywana wartość: 638.4268188476562
Rzeczywista wartość: 854.9099731445312
Przykład 8:
Przewidywana

# Model podstawowy, ale bez dropout'u - chyba najlepszy

In [53]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GraphConv

class GraphRegressionModel(nn.Module):
    def __init__(self, num_node_features=1, hidden_dim=64, output_dim=1):
        super(GraphRegressionModel, self).__init__()

        # Graph Convolutional Layer
        self.conv1 = GraphConv(num_node_features, hidden_dim)

        # Fully Connected Layers (without Dropout)
        self.fc1 = nn.Linear(hidden_dim + 2, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index, edge_weight, global_feature = data.x, data.edge_index, data.edge_weight, data.global_feature

        # Apply Graph Convolution
        x = self.conv1(x, edge_index, edge_weight=edge_weight)

        # Global features concatenation
        global_feature = global_feature.expand(x.size(0), -1)
        x = torch.cat([x, global_feature], dim=1)

        # Fully Connected Layers (without Dropout)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x
    
    def loss(self, pred, score):
        return F.mse_loss(pred, score)


# Zbieranie danych

In [54]:
from hyperopt import fmin, tpe, hp

# Zdefiniuj funkcję oceny (score function)
def objective(params):
    hidden_dim = int(params['hidden_dim'])
    #dropout = params['dropout']

    model = GraphRegressionModel(hidden_dim=hidden_dim).to(device)
    # Definiuj optymalizator, liczbę epok, itp.
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    num_epochs = 200
    
    for epoch in range(num_epochs):
        # Trening i ewaluacja modelu
        for data in train_loader:
            data.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = model.loss(output, data.y.view(-1, 1).float())
            loss.backward()
            optimizer.step()
    
    # Ewaluacja na zbiorze testowym
    test_loss = 0.0  
    for data in test_loader:
        data.to(device)
        output = model(data)
        test_loss += model.loss(output, data.y.view(-1, 1).float()).item()
    
    # Zwróć funkcję oceny (score)
    return test_loss

# Przestrzeń poszukiwań dla hyperopt
space = {
    'hidden_dim': hp.quniform('hidden_dim', 32, 256, 32),  # Przeszukuj wartości co 32
   # 'dropout': hp.uniform('dropout', 0.05, 0.8),
}

# Minimalizacja funkcji oceny za pomocą algorytmu TPE
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10)  # Dla przykładu ustawiono max_evals na 10
print("Najlepsze hiperparametry:", best)
# Open the file in 'a' mode (append mode)

import torch

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Ustawienie optymalnych hiperparametrów
optimal_hidden_dim = int(best['hidden_dim'])
#optimal_dropout = best['dropout']

#optimal_hidden_dim = 192
#optimal_dropout = 0.06

# Inicjalizacja modelu z optymalnymi hiperparametrami
optimal_model = GraphRegressionModel(hidden_dim=optimal_hidden_dim).to(device)



# Definiuj optymalizator, liczbę epok, itp.
optimizer = torch.optim.Adam(optimal_model.parameters(), lr=0.001)
num_epochs = 100

# Listy do śledzenia train loss i test loss
train_losses = []
test_losses = []

# Trening modelu
for epoch in range(num_epochs):
    if epoch % 10 == 0:
        print("Epoch:", epoch)
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = optimal_model(data)
        loss = optimal_model.loss(output, data.y.view(-1, 1).float())
        loss.backward()
        optimizer.step()

sample_data = test_dataset[0].to(device)

# Przeprowadzenie predykcji na przykładowym obiekcie
optimal_model.eval()
with torch.no_grad():
    prediction = optimal_model(sample_data)[0][0].item()

# Wyświetlenie wyników
print("Przewidywana wartość:", prediction)
print("Rzeczywista wartość:", sample_data.y.item())

total_accuracy = 0.0

# Iteracja przez cały zbiór testowy
for i, sample_data in enumerate(test_dataset):
    # Przeprowadzenie predykcji na przykładowym obiekcie
    optimal_model.eval()
    with torch.no_grad():
        prediction = optimal_model(sample_data.to(device))[0][0].item()
    
    # Wyświetlenie wyników
    print(f"Przykład {i + 1}:")
    print("Przewidywana wartość:", prediction)
    print("Rzeczywista wartość:", sample_data.y.item())
    
    # Obliczenie procentowej jakości regresji
    actual_value = sample_data.y.item()
    accuracy = 100 * (1 - abs(prediction - actual_value) / actual_value)
    total_accuracy += accuracy
    
    # print(f"Procentowa jakość regresji: {accuracy:.2f}%\n")

# Obliczenie średniej jakości procentowej
average_accuracy = total_accuracy / len(test_dataset)
print(f"Średnia procentowa jakość regresji dla wszystkich przykładów: {average_accuracy:.2f}%")

from sklearn.metrics import r2_score, mean_squared_error

# Załóżmy, że y_true i y_pred to twoje rzeczywiste i przewidywane wartości.
#optimal_model.load_state_dict(torch.load("model_big"))

total_accuracy = 0.0
y_true = []
y_pred = []

# Iteracja przez cały zbiór testowy
for i, sample_data in enumerate(test_dataset):
    # Przeprowadzenie predykcji na przykładowym obiekcie
    optimal_model.eval()
    with torch.no_grad():
        prediction = optimal_model(sample_data)[0][0].item()
        y_pred.append(prediction)
    
    y_true.append(sample_data.y.item())
    
    # Wyświetlenie wyników
    # print(f"Przykład {i + 1}:")
    # print("Przewidywana wartość:", prediction)
    # print("Rzeczywista wartość:", sample_data.y.item())
    
    # Obliczenie procentowej jakości regresji
    actual_value = sample_data.y.item()
    accuracy = 100 * (1 - abs(prediction - actual_value) / actual_value)
    total_accuracy += accuracy
    
    # print(f"Procentowa jakość regresji: {accuracy:.2f}%\n")

# Obliczenie średniej jakości procentowej
average_accuracy = total_accuracy / len(test_dataset)
r2 = r2_score(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
rmse = mse**(1/2)

print(f"R2 Score = {r2:.4f}")
print(f"MSE Score = {mse:.4f}")
print(f"RMSE Score = {rmse:.4f}")

y_mean = sum(y_true) / len(y_true)
print(f"Jaki procent zmiennej zależnej stanowi błąd RMSE? {rmse / y_mean * 100:.4f}%")

with open("file.txt", "a") as file:
    # Append the text to the file
    file.write("\n" + "ModelPostawowybezDrop")  # Add a new line before appending
    file.write("\n" + "R2 "+ str(r2) )
    file.write("\n" + "MSE "+ str(mse))
    file.write("\n" + "RMSE "+ str(rmse))
    file.write("\n" + "Awerage "+ str(average_accuracy))

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/tmp/ipykernel_4183/4232110703.py:34: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([28, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(pred, score)



100%|██████████| 10/10 [08:46<00:00, 52.62s/trial, best loss: 24540.59095683694]
Najlepsze hiperparametry: {'hidden_dim': 224.0}
Epoch: 0
Epoch: 10
Epoch: 20
Epoch: 30
Epoch: 40
Epoch: 50
Epoch: 60
Epoch: 70
Epoch: 80
Epoch: 90
Przewidywana wartość: 703.48779296875
Rzeczywista wartość: 874.5499877929688
Przykład 1:
Przewidywana wartość: 703.48779296875
Rzeczywista wartość: 874.5499877929688
Przykład 2:
Przewidywana wartość: 607.0521240234375
Rzeczywista wartość: 569.8499755859375
Przykład 3:
Przewidywana wartość: 563.8634033203125
Rzeczywista wartość: 450.54998779296875
Przykład 4:
Przewidywana wartość: 721.365966796875
Rzeczywista wartość: 839.4600219726562
Przykład 5:
Przewidywana wartość: 654.2081909179688
Rzeczywista wartość: 705.010009765625
Przykład 6:
Przewidywana wartość: 648.4389038085938
Rzeczywista wartość: 697.9400024414062
Przykład 7:
Przewidywana wartość: 717.0652465820312
Rzeczywista wartość: 854.9099731445312
Przykład 8:
Przewidywana wartość: 690.7364501953125
Rzeczywis

# Model podstawowy bez cech globalnych

In [65]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GraphConv

class GraphRegressionModel(nn.Module):
    def __init__(self, num_node_features=1, hidden_dim=64, output_dim=1, dropout=0.5):
        super(GraphRegressionModel, self).__init__()

        # Graph Convolutional Layer
        self.conv1 = GraphConv(num_node_features, hidden_dim)

        # Fully Connected Layers with Dropout
        self.fc1 = nn.Linear(hidden_dim, hidden_dim)
        self.dropout = nn.Dropout(p=dropout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index, edge_weight = data.x, data.edge_index, data.edge_weight

        # Apply Graph Convolution
        x = self.conv1(x, edge_index, edge_weight=edge_weight)

        # Fully Connected Layers with Dropout
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x
    
    def loss(self, pred, score):
        return F.mse_loss(pred, score)


# Zbieranie danych

In [66]:
from hyperopt import fmin, tpe, hp

# Zdefiniuj funkcję oceny (score function)
def objective(params):
    hidden_dim = int(params['hidden_dim'])
    dropout = params['dropout']

    model = GraphRegressionModel(hidden_dim=hidden_dim, dropout=dropout).to(device)
    # Definiuj optymalizator, liczbę epok, itp.
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    num_epochs = 200
    
    for epoch in range(num_epochs):
        # Trening i ewaluacja modelu
        for data in train_loader:
            data.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = model.loss(output, data.y.view(-1, 1).float())
            loss.backward()
            optimizer.step()
    
    # Ewaluacja na zbiorze testowym
    test_loss = 0.0  
    for data in test_loader:
        data.to(device)
        output = model(data)
        test_loss += model.loss(output, data.y.view(-1, 1).float()).item()
    
    # Zwróć funkcję oceny (score)
    return test_loss

# Przestrzeń poszukiwań dla hyperopt
space = {
    'hidden_dim': hp.quniform('hidden_dim', 32, 256, 32),  # Przeszukuj wartości co 32
    'dropout': hp.uniform('dropout', 0.05, 0.8),
}

# Minimalizacja funkcji oceny za pomocą algorytmu TPE
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10)  # Dla przykładu ustawiono max_evals na 10
print("Najlepsze hiperparametry:", best)
# Open the file in 'a' mode (append mode)

import torch

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Ustawienie optymalnych hiperparametrów
optimal_hidden_dim = int(best['hidden_dim'])
optimal_dropout = best['dropout']

#optimal_hidden_dim = 192
#optimal_dropout = 0.06

# Inicjalizacja modelu z optymalnymi hiperparametrami
optimal_model = GraphRegressionModel(hidden_dim=optimal_hidden_dim, dropout=optimal_dropout).to(device)



# Definiuj optymalizator, liczbę epok, itp.
optimizer = torch.optim.Adam(optimal_model.parameters(), lr=0.001)
num_epochs = 100

# Listy do śledzenia train loss i test loss
train_losses = []
test_losses = []

# Trening modelu
for epoch in range(num_epochs):
    if epoch % 10 == 0:
        print("Epoch:", epoch)
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = optimal_model(data)
        loss = optimal_model.loss(output, data.y.view(-1, 1).float())
        loss.backward()
        optimizer.step()

sample_data = test_dataset[0].to(device)

# Przeprowadzenie predykcji na przykładowym obiekcie
optimal_model.eval()
with torch.no_grad():
    prediction = optimal_model(sample_data)[0][0].item()

# Wyświetlenie wyników
print("Przewidywana wartość:", prediction)
print("Rzeczywista wartość:", sample_data.y.item())

total_accuracy = 0.0

# Iteracja przez cały zbiór testowy
for i, sample_data in enumerate(test_dataset):
    # Przeprowadzenie predykcji na przykładowym obiekcie
    optimal_model.eval()
    with torch.no_grad():
        prediction = optimal_model(sample_data.to(device))[0][0].item()
    
    # Wyświetlenie wyników
    print(f"Przykład {i + 1}:")
    print("Przewidywana wartość:", prediction)
    print("Rzeczywista wartość:", sample_data.y.item())
    
    # Obliczenie procentowej jakości regresji
    actual_value = sample_data.y.item()
    accuracy = 100 * (1 - abs(prediction - actual_value) / actual_value)
    total_accuracy += accuracy
    
    # print(f"Procentowa jakość regresji: {accuracy:.2f}%\n")

# Obliczenie średniej jakości procentowej
average_accuracy = total_accuracy / len(test_dataset)
print(f"Średnia procentowa jakość regresji dla wszystkich przykładów: {average_accuracy:.2f}%")

from sklearn.metrics import r2_score, mean_squared_error

# Załóżmy, że y_true i y_pred to twoje rzeczywiste i przewidywane wartości.
#optimal_model.load_state_dict(torch.load("model_big"))

total_accuracy = 0.0
y_true = []
y_pred = []

# Iteracja przez cały zbiór testowy
for i, sample_data in enumerate(test_dataset):
    # Przeprowadzenie predykcji na przykładowym obiekcie
    optimal_model.eval()
    with torch.no_grad():
        prediction = optimal_model(sample_data)[0][0].item()
        y_pred.append(prediction)
    
    y_true.append(sample_data.y.item())
    
    # Wyświetlenie wyników
    # print(f"Przykład {i + 1}:")
    # print("Przewidywana wartość:", prediction)
    # print("Rzeczywista wartość:", sample_data.y.item())
    
    # Obliczenie procentowej jakości regresji
    actual_value = sample_data.y.item()
    accuracy = 100 * (1 - abs(prediction - actual_value) / actual_value)
    total_accuracy += accuracy
    
    # print(f"Procentowa jakość regresji: {accuracy:.2f}%\n")

# Obliczenie średniej jakości procentowej
average_accuracy = total_accuracy / len(test_dataset)
r2 = r2_score(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
rmse = mse**(1/2)

print(f"R2 Score = {r2:.4f}")
print(f"MSE Score = {mse:.4f}")
print(f"RMSE Score = {rmse:.4f}")

y_mean = sum(y_true) / len(y_true)
print(f"Jaki procent zmiennej zależnej stanowi błąd RMSE? {rmse / y_mean * 100:.4f}%")

with open("file.txt", "a") as file:
    # Append the text to the file
    file.write("\n" + "ModelPodstawowyBezCech")  # Add a new line before appending
    file.write("\n" + "R2 "+ str(r2) )
    file.write("\n" + "MSE "+ str(mse))
    file.write("\n" + "RMSE "+ str(rmse))
    file.write("\n" + "Awerage "+ str(average_accuracy))

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/tmp/ipykernel_4183/1320226180.py:32: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([28, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(pred, score)



100%|██████████| 10/10 [09:15<00:00, 55.56s/trial, best loss: 2903893.548400879]
Najlepsze hiperparametry: {'dropout': 0.09201551657903285, 'hidden_dim': 224.0}
Epoch: 0
Epoch: 10
Epoch: 20
Epoch: 30
Epoch: 40
Epoch: 50
Epoch: 60
Epoch: 70
Epoch: 80
Epoch: 90
Przewidywana wartość: 651.00830078125
Rzeczywista wartość: 874.5499877929688
Przykład 1:
Przewidywana wartość: 651.00830078125
Rzeczywista wartość: 874.5499877929688
Przykład 2:
Przewidywana wartość: 651.00830078125
Rzeczywista wartość: 569.8499755859375
Przykład 3:
Przewidywana wartość: 651.00830078125
Rzeczywista wartość: 450.54998779296875
Przykład 4:
Przewidywana wartość: 651.00830078125
Rzeczywista wartość: 839.4600219726562
Przykład 5:
Przewidywana wartość: 651.00830078125
Rzeczywista wartość: 705.010009765625
Przykład 6:
Przewidywana wartość: 651.00830078125
Rzeczywista wartość: 697.9400024414062
Przykład 7:
Przewidywana wartość: 651.00830078125
Rzeczywista wartość: 854.9099731445312
Przykład 8:
Przewidywana wartość: 651.00

# INNE MODELE

# ChGW1

In [57]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GraphRegressionModelGCN(nn.Module):
    def __init__(self, num_node_features=1, hidden_dim=64, output_dim=1, dropout=0.5):
        super(GraphRegressionModelGCN, self).__init__()

        # Graph Convolutional Layer using GCNConv
        self.conv1 = GCNConv(num_node_features, hidden_dim)

        # Fully Connected Layers with Dropout
        self.fc1 = nn.Linear(hidden_dim + 2, hidden_dim)
        self.dropout = nn.Dropout(p=dropout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index, edge_weight, global_feature = data.x, data.edge_index, data.edge_weight, data.global_feature

        # Apply Graph Convolution using GCNConv
        x = self.conv1(x, edge_index, edge_weight=edge_weight)

        # Global features concatenation
        global_feature = global_feature.expand(x.size(0), -1)
        x = torch.cat([x, global_feature], dim=1)

        # Fully Connected Layers with Dropout
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x
    
    def loss(self, pred, score):
        return F.mse_loss(pred, score)

# Zbieranie danych

In [58]:
from hyperopt import fmin, tpe, hp

# Zdefiniuj funkcję oceny (score function)
def objective(params):
    hidden_dim = int(params['hidden_dim'])
    dropout = params['dropout']

    model = GraphRegressionModel(hidden_dim=hidden_dim, dropout=dropout).to(device)
    # Definiuj optymalizator, liczbę epok, itp.
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    num_epochs = 200
    
    for epoch in range(num_epochs):
        # Trening i ewaluacja modelu
        for data in train_loader:
            data.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = model.loss(output, data.y.view(-1, 1).float())
            loss.backward()
            optimizer.step()
    
    # Ewaluacja na zbiorze testowym
    test_loss = 0.0  
    for data in test_loader:
        data.to(device)
        output = model(data)
        test_loss += model.loss(output, data.y.view(-1, 1).float()).item()
    
    # Zwróć funkcję oceny (score)
    return test_loss

# Przestrzeń poszukiwań dla hyperopt
space = {
    'hidden_dim': hp.quniform('hidden_dim', 32, 256, 32),  # Przeszukuj wartości co 32
    'dropout': hp.uniform('dropout', 0.05, 0.8),
}

# Minimalizacja funkcji oceny za pomocą algorytmu TPE
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10)  # Dla przykładu ustawiono max_evals na 10
print("Najlepsze hiperparametry:", best)
# Open the file in 'a' mode (append mode)

import torch

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Ustawienie optymalnych hiperparametrów
optimal_hidden_dim = int(best['hidden_dim'])
optimal_dropout = best['dropout']

#optimal_hidden_dim = 192
#optimal_dropout = 0.06

# Inicjalizacja modelu z optymalnymi hiperparametrami
optimal_model = GraphRegressionModel(hidden_dim=optimal_hidden_dim, dropout=optimal_dropout).to(device)



# Definiuj optymalizator, liczbę epok, itp.
optimizer = torch.optim.Adam(optimal_model.parameters(), lr=0.001)
num_epochs = 100

# Listy do śledzenia train loss i test loss
train_losses = []
test_losses = []

# Trening modelu
for epoch in range(num_epochs):
    if epoch % 10 == 0:
        print("Epoch:", epoch)
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = optimal_model(data)
        loss = optimal_model.loss(output, data.y.view(-1, 1).float())
        loss.backward()
        optimizer.step()

sample_data = test_dataset[0].to(device)

# Przeprowadzenie predykcji na przykładowym obiekcie
optimal_model.eval()
with torch.no_grad():
    prediction = optimal_model(sample_data)[0][0].item()

# Wyświetlenie wyników
print("Przewidywana wartość:", prediction)
print("Rzeczywista wartość:", sample_data.y.item())

total_accuracy = 0.0

# Iteracja przez cały zbiór testowy
for i, sample_data in enumerate(test_dataset):
    # Przeprowadzenie predykcji na przykładowym obiekcie
    optimal_model.eval()
    with torch.no_grad():
        prediction = optimal_model(sample_data.to(device))[0][0].item()
    
    # Wyświetlenie wyników
    print(f"Przykład {i + 1}:")
    print("Przewidywana wartość:", prediction)
    print("Rzeczywista wartość:", sample_data.y.item())
    
    # Obliczenie procentowej jakości regresji
    actual_value = sample_data.y.item()
    accuracy = 100 * (1 - abs(prediction - actual_value) / actual_value)
    total_accuracy += accuracy
    
    # print(f"Procentowa jakość regresji: {accuracy:.2f}%\n")

# Obliczenie średniej jakości procentowej
average_accuracy = total_accuracy / len(test_dataset)
print(f"Średnia procentowa jakość regresji dla wszystkich przykładów: {average_accuracy:.2f}%")

from sklearn.metrics import r2_score, mean_squared_error

# Załóżmy, że y_true i y_pred to twoje rzeczywiste i przewidywane wartości.
#optimal_model.load_state_dict(torch.load("model_big"))

total_accuracy = 0.0
y_true = []
y_pred = []

# Iteracja przez cały zbiór testowy
for i, sample_data in enumerate(test_dataset):
    # Przeprowadzenie predykcji na przykładowym obiekcie
    optimal_model.eval()
    with torch.no_grad():
        prediction = optimal_model(sample_data)[0][0].item()
        y_pred.append(prediction)
    
    y_true.append(sample_data.y.item())
    
    # Wyświetlenie wyników
    # print(f"Przykład {i + 1}:")
    # print("Przewidywana wartość:", prediction)
    # print("Rzeczywista wartość:", sample_data.y.item())
    
    # Obliczenie procentowej jakości regresji
    actual_value = sample_data.y.item()
    accuracy = 100 * (1 - abs(prediction - actual_value) / actual_value)
    total_accuracy += accuracy
    
    # print(f"Procentowa jakość regresji: {accuracy:.2f}%\n")

# Obliczenie średniej jakości procentowej
average_accuracy = total_accuracy / len(test_dataset)
r2 = r2_score(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
rmse = mse**(1/2)

print(f"R2 Score = {r2:.4f}")
print(f"MSE Score = {mse:.4f}")
print(f"RMSE Score = {rmse:.4f}")

y_mean = sum(y_true) / len(y_true)
print(f"Jaki procent zmiennej zależnej stanowi błąd RMSE? {rmse / y_mean * 100:.4f}%")

with open("file.txt", "a") as file:
    # Append the text to the file
    file.write("\n" + "ChGW1")  # Add a new line before appending
    file.write("\n" + "R2 "+ str(r2) )
    file.write("\n" + "MSE "+ str(mse))
    file.write("\n" + "RMSE "+ str(rmse))
    file.write("\n" + "Awerage "+ str(average_accuracy))

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/tmp/ipykernel_4183/1320226180.py:32: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([28, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(pred, score)



100%|██████████| 10/10 [09:01<00:00, 54.16s/trial, best loss: 2916286.6631469727]
Najlepsze hiperparametry: {'dropout': 0.07658456297831713, 'hidden_dim': 224.0}
Epoch: 0
Epoch: 10
Epoch: 20
Epoch: 30
Epoch: 40
Epoch: 50
Epoch: 60
Epoch: 70
Epoch: 80
Epoch: 90
Przewidywana wartość: 637.968505859375
Rzeczywista wartość: 874.5499877929688
Przykład 1:
Przewidywana wartość: 637.968505859375
Rzeczywista wartość: 874.5499877929688
Przykład 2:
Przewidywana wartość: 637.968505859375
Rzeczywista wartość: 569.8499755859375
Przykład 3:
Przewidywana wartość: 637.968505859375
Rzeczywista wartość: 450.54998779296875
Przykład 4:
Przewidywana wartość: 637.968505859375
Rzeczywista wartość: 839.4600219726562
Przykład 5:
Przewidywana wartość: 637.968505859375
Rzeczywista wartość: 705.010009765625
Przykład 6:
Przewidywana wartość: 637.968505859375
Rzeczywista wartość: 697.9400024414062
Przykład 7:
Przewidywana wartość: 637.968505859375
Rzeczywista wartość: 854.9099731445312
Przykład 8:
Przewidywana wartoś

# ChGW2

In [59]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import ChebConv

class GraphRegressionModelCheb(nn.Module):
    def __init__(self, num_node_features=1, hidden_dim=64, output_dim=1, dropout=0.5):
        super(GraphRegressionModelCheb, self).__init__()

        # Chebyshev Graph Convolutional Layer
        self.conv1 = ChebConv(num_node_features, hidden_dim, K=2)  # K is the filter size (you can adjust it)

        # Fully Connected Layers with Dropout
        self.fc1 = nn.Linear(hidden_dim + 2, hidden_dim)
        self.dropout = nn.Dropout(p=dropout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index, edge_weight, global_feature = data.x, data.edge_index, data.edge_weight, data.global_feature

        # Apply Chebyshev Graph Convolution
        x = self.conv1(x, edge_index, edge_weight=edge_weight)

        # Global features concatenation
        global_feature = global_feature.expand(x.size(0), -1)
        x = torch.cat([x, global_feature], dim=1)

        # Fully Connected Layers with Dropout
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x
    
    def loss(self, pred, score):
        return F.mse_loss(pred, score)


# Zbieranie danych

In [60]:

from hyperopt import fmin, tpe, hp

# Zdefiniuj funkcję oceny (score function)
def objective(params):
    hidden_dim = int(params['hidden_dim'])
    dropout = params['dropout']

    model = GraphRegressionModel(hidden_dim=hidden_dim, dropout=dropout).to(device)
    # Definiuj optymalizator, liczbę epok, itp.
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    num_epochs = 200
    
    for epoch in range(num_epochs):
        # Trening i ewaluacja modelu
        for data in train_loader:
            data.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = model.loss(output, data.y.view(-1, 1).float())
            loss.backward()
            optimizer.step()
    
    # Ewaluacja na zbiorze testowym
    test_loss = 0.0  
    for data in test_loader:
        data.to(device)
        output = model(data)
        test_loss += model.loss(output, data.y.view(-1, 1).float()).item()
    
    # Zwróć funkcję oceny (score)
    return test_loss

# Przestrzeń poszukiwań dla hyperopt
space = {
    'hidden_dim': hp.quniform('hidden_dim', 32, 256, 32),  # Przeszukuj wartości co 32
    'dropout': hp.uniform('dropout', 0.05, 0.8),
}

# Minimalizacja funkcji oceny za pomocą algorytmu TPE
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10)  # Dla przykładu ustawiono max_evals na 10
print("Najlepsze hiperparametry:", best)
# Open the file in 'a' mode (append mode)

import torch

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Ustawienie optymalnych hiperparametrów
optimal_hidden_dim = int(best['hidden_dim'])
optimal_dropout = best['dropout']

#optimal_hidden_dim = 192
#optimal_dropout = 0.06

# Inicjalizacja modelu z optymalnymi hiperparametrami
optimal_model = GraphRegressionModel(hidden_dim=optimal_hidden_dim, dropout=optimal_dropout).to(device)



# Definiuj optymalizator, liczbę epok, itp.
optimizer = torch.optim.Adam(optimal_model.parameters(), lr=0.001)
num_epochs = 100

# Listy do śledzenia train loss i test loss
train_losses = []
test_losses = []

# Trening modelu
for epoch in range(num_epochs):
    if epoch % 10 == 0:
        print("Epoch:", epoch)
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = optimal_model(data)
        loss = optimal_model.loss(output, data.y.view(-1, 1).float())
        loss.backward()
        optimizer.step()

sample_data = test_dataset[0].to(device)

# Przeprowadzenie predykcji na przykładowym obiekcie
optimal_model.eval()
with torch.no_grad():
    prediction = optimal_model(sample_data)[0][0].item()

# Wyświetlenie wyników
print("Przewidywana wartość:", prediction)
print("Rzeczywista wartość:", sample_data.y.item())

total_accuracy = 0.0

# Iteracja przez cały zbiór testowy
for i, sample_data in enumerate(test_dataset):
    # Przeprowadzenie predykcji na przykładowym obiekcie
    optimal_model.eval()
    with torch.no_grad():
        prediction = optimal_model(sample_data.to(device))[0][0].item()
    
    # Wyświetlenie wyników
    print(f"Przykład {i + 1}:")
    print("Przewidywana wartość:", prediction)
    print("Rzeczywista wartość:", sample_data.y.item())
    
    # Obliczenie procentowej jakości regresji
    actual_value = sample_data.y.item()
    accuracy = 100 * (1 - abs(prediction - actual_value) / actual_value)
    total_accuracy += accuracy
    
    # print(f"Procentowa jakość regresji: {accuracy:.2f}%\n")

# Obliczenie średniej jakości procentowej
average_accuracy = total_accuracy / len(test_dataset)
print(f"Średnia procentowa jakość regresji dla wszystkich przykładów: {average_accuracy:.2f}%")

from sklearn.metrics import r2_score, mean_squared_error

# Załóżmy, że y_true i y_pred to twoje rzeczywiste i przewidywane wartości.
#optimal_model.load_state_dict(torch.load("model_big"))

total_accuracy = 0.0
y_true = []
y_pred = []

# Iteracja przez cały zbiór testowy
for i, sample_data in enumerate(test_dataset):
    # Przeprowadzenie predykcji na przykładowym obiekcie
    optimal_model.eval()
    with torch.no_grad():
        prediction = optimal_model(sample_data)[0][0].item()
        y_pred.append(prediction)
    
    y_true.append(sample_data.y.item())
    
    # Wyświetlenie wyników
    # print(f"Przykład {i + 1}:")
    # print("Przewidywana wartość:", prediction)
    # print("Rzeczywista wartość:", sample_data.y.item())
    
    # Obliczenie procentowej jakości regresji
    actual_value = sample_data.y.item()
    accuracy = 100 * (1 - abs(prediction - actual_value) / actual_value)
    total_accuracy += accuracy
    
    # print(f"Procentowa jakość regresji: {accuracy:.2f}%\n")

# Obliczenie średniej jakości procentowej
average_accuracy = total_accuracy / len(test_dataset)
r2 = r2_score(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
rmse = mse**(1/2)

print(f"R2 Score = {r2:.4f}")
print(f"MSE Score = {mse:.4f}")
print(f"RMSE Score = {rmse:.4f}")

y_mean = sum(y_true) / len(y_true)
print(f"Jaki procent zmiennej zależnej stanowi błąd RMSE? {rmse / y_mean * 100:.4f}%")

with open("file.txt", "a") as file:
    # Append the text to the file
    file.write("\n" + "ChGW2")  # Add a new line before appending
    file.write("\n" + "R2 "+ str(r2) )
    file.write("\n" + "MSE "+str( mse))
    file.write("\n" + "RMSE "+ str(rmse))
    file.write("\n" + "Awerage "+ str(average_accuracy))

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/tmp/ipykernel_4183/1320226180.py:32: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([28, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(pred, score)



100%|██████████| 10/10 [09:16<00:00, 55.66s/trial, best loss: 2872102.585144043]
Najlepsze hiperparametry: {'dropout': 0.1027999314932788, 'hidden_dim': 160.0}
Epoch: 0
Epoch: 10
Epoch: 20
Epoch: 30
Epoch: 40
Epoch: 50
Epoch: 60
Epoch: 70
Epoch: 80
Epoch: 90
Przewidywana wartość: 636.5919799804688
Rzeczywista wartość: 874.5499877929688
Przykład 1:
Przewidywana wartość: 636.5919799804688
Rzeczywista wartość: 874.5499877929688
Przykład 2:
Przewidywana wartość: 636.5919799804688
Rzeczywista wartość: 569.8499755859375
Przykład 3:
Przewidywana wartość: 636.5919799804688
Rzeczywista wartość: 450.54998779296875
Przykład 4:
Przewidywana wartość: 636.5919799804688
Rzeczywista wartość: 839.4600219726562
Przykład 5:
Przewidywana wartość: 636.5919799804688
Rzeczywista wartość: 705.010009765625
Przykład 6:
Przewidywana wartość: 636.5919799804688
Rzeczywista wartość: 697.9400024414062
Przykład 7:
Przewidywana wartość: 636.5919799804688
Rzeczywista wartość: 854.9099731445312
Przykład 8:
Przewidywana 

# EdgeConv

In [61]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GraphConv, EdgeConv

class CustomEdgeConv(nn.Module):
    def __init__(self, num_node_features, hidden_dim):
        super(CustomEdgeConv, self).__init__()
        self.layers = nn.ModuleList([
            nn.Linear(1, 32),  # Adjust the input size for the first linear layer
            nn.ReLU(),
            nn.Linear(32, hidden_dim)
        ])

    def forward(self, x, edge_index):
        for layer in self.layers:
            x = layer(x)
        return x

class GraphRegressionModel(nn.Module):
    def __init__(self, num_node_features=1, hidden_dim=64, output_dim=1, dropout=0.5):
        super(GraphRegressionModel, self).__init__()

        # Graph Convolutional Layer
        #self.conv1 = GraphConv(num_node_features, hidden_dim)

        # Edge Convolutional Layer
        self.conv2 = CustomEdgeConv(num_node_features, hidden_dim+2)

        # Fully Connected Layers with Dropout
        self.fc1 = nn.Linear(hidden_dim + 4, hidden_dim)
        self.dropout = nn.Dropout(p=dropout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index, edge_weight, global_feature = data.x, data.edge_index, data.edge_weight, data.global_feature

        # Apply Graph Convolution
        #x = self.conv1(x, edge_index, edge_weight=edge_weight)
        # Apply Edge Convolution
        x = self.conv2(x, edge_index)

        # Global features concatenation
        global_feature = global_feature.expand(x.size(0), -1)
        x = torch.cat([x, global_feature], dim=1)

        # Fully Connected Layers with Dropout
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x

    def loss(self, pred, score):
        return F.mse_loss(pred, score)

# Zbieranie danych

In [62]:
from hyperopt import fmin, tpe, hp

# Zdefiniuj funkcję oceny (score function)
def objective(params):
    hidden_dim = int(params['hidden_dim'])
    dropout = params['dropout']

    model = GraphRegressionModel(hidden_dim=hidden_dim, dropout=dropout).to(device)
    # Definiuj optymalizator, liczbę epok, itp.
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    num_epochs = 200
    
    for epoch in range(num_epochs):
        # Trening i ewaluacja modelu
        for data in train_loader:
            data.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = model.loss(output, data.y.view(-1, 1).float())
            loss.backward()
            optimizer.step()
    
    # Ewaluacja na zbiorze testowym
    test_loss = 0.0  
    for data in test_loader:
        data.to(device)
        output = model(data)
        test_loss += model.loss(output, data.y.view(-1, 1).float()).item()
    
    # Zwróć funkcję oceny (score)
    return test_loss

# Przestrzeń poszukiwań dla hyperopt
space = {
    'hidden_dim': hp.quniform('hidden_dim', 32, 256, 32),  # Przeszukuj wartości co 32
    'dropout': hp.uniform('dropout', 0.05, 0.8),
}

# Minimalizacja funkcji oceny za pomocą algorytmu TPE
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10)  # Dla przykładu ustawiono max_evals na 10
print("Najlepsze hiperparametry:", best)
# Open the file in 'a' mode (append mode)

import torch

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Ustawienie optymalnych hiperparametrów
optimal_hidden_dim = int(best['hidden_dim'])
optimal_dropout = best['dropout']

#optimal_hidden_dim = 192
#optimal_dropout = 0.06

# Inicjalizacja modelu z optymalnymi hiperparametrami
optimal_model = GraphRegressionModel(hidden_dim=optimal_hidden_dim, dropout=optimal_dropout).to(device)



# Definiuj optymalizator, liczbę epok, itp.
optimizer = torch.optim.Adam(optimal_model.parameters(), lr=0.001)
num_epochs = 100

# Listy do śledzenia train loss i test loss
train_losses = []
test_losses = []

# Trening modelu
for epoch in range(num_epochs):
    if epoch % 10 == 0:
        print("Epoch:", epoch)
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = optimal_model(data)
        loss = optimal_model.loss(output, data.y.view(-1, 1).float())
        loss.backward()
        optimizer.step()

sample_data = test_dataset[0].to(device)

# Przeprowadzenie predykcji na przykładowym obiekcie
optimal_model.eval()
with torch.no_grad():
    prediction = optimal_model(sample_data)[0][0].item()

# Wyświetlenie wyników
print("Przewidywana wartość:", prediction)
print("Rzeczywista wartość:", sample_data.y.item())

total_accuracy = 0.0

# Iteracja przez cały zbiór testowy
for i, sample_data in enumerate(test_dataset):
    # Przeprowadzenie predykcji na przykładowym obiekcie
    optimal_model.eval()
    with torch.no_grad():
        prediction = optimal_model(sample_data.to(device))[0][0].item()
    
    # Wyświetlenie wyników
    print(f"Przykład {i + 1}:")
    print("Przewidywana wartość:", prediction)
    print("Rzeczywista wartość:", sample_data.y.item())
    
    # Obliczenie procentowej jakości regresji
    actual_value = sample_data.y.item()
    accuracy = 100 * (1 - abs(prediction - actual_value) / actual_value)
    total_accuracy += accuracy
    
    # print(f"Procentowa jakość regresji: {accuracy:.2f}%\n")

# Obliczenie średniej jakości procentowej
average_accuracy = total_accuracy / len(test_dataset)
print(f"Średnia procentowa jakość regresji dla wszystkich przykładów: {average_accuracy:.2f}%")

from sklearn.metrics import r2_score, mean_squared_error

# Załóżmy, że y_true i y_pred to twoje rzeczywiste i przewidywane wartości.
#optimal_model.load_state_dict(torch.load("model_big"))

total_accuracy = 0.0
y_true = []
y_pred = []

# Iteracja przez cały zbiór testowy
for i, sample_data in enumerate(test_dataset):
    # Przeprowadzenie predykcji na przykładowym obiekcie
    optimal_model.eval()
    with torch.no_grad():
        prediction = optimal_model(sample_data)[0][0].item()
        y_pred.append(prediction)
    
    y_true.append(sample_data.y.item())
    
    # Wyświetlenie wyników
    # print(f"Przykład {i + 1}:")
    # print("Przewidywana wartość:", prediction)
    # print("Rzeczywista wartość:", sample_data.y.item())
    
    # Obliczenie procentowej jakości regresji
    actual_value = sample_data.y.item()
    accuracy = 100 * (1 - abs(prediction - actual_value) / actual_value)
    total_accuracy += accuracy
    
    # print(f"Procentowa jakość regresji: {accuracy:.2f}%\n")

# Obliczenie średniej jakości procentowej
average_accuracy = total_accuracy / len(test_dataset)
r2 = r2_score(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
rmse = mse**(1/2)

print(f"R2 Score = {r2:.4f}")
print(f"MSE Score = {mse:.4f}")
print(f"RMSE Score = {rmse:.4f}")

y_mean = sum(y_true) / len(y_true)
print(f"Jaki procent zmiennej zależnej stanowi błąd RMSE? {rmse / y_mean * 100:.4f}%")

with open("file.txt", "a") as file:
    # Append the text to the file
    file.write("\n" + "EdgeConv")  # Add a new line before appending
    file.write("\n" + "R2 "+ str(r2) )
    file.write("\n" + "MSE "+ str(mse))
    file.write("\n" + "RMSE "+ str(rmse))
    file.write("\n" + "Awerage "+ str(average_accuracy))

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/tmp/ipykernel_4183/1014723709.py:55: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([28, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(pred, score)



100%|██████████| 10/10 [08:54<00:00, 53.49s/trial, best loss: 139210.4389038086]
Najlepsze hiperparametry: {'dropout': 0.28583917405778647, 'hidden_dim': 224.0}
Epoch: 0
Epoch: 10
Epoch: 20
Epoch: 30
Epoch: 40
Epoch: 50
Epoch: 60
Epoch: 70
Epoch: 80
Epoch: 90
Przewidywana wartość: 670.9161987304688
Rzeczywista wartość: 874.5499877929688
Przykład 1:
Przewidywana wartość: 670.9161987304688
Rzeczywista wartość: 874.5499877929688
Przykład 2:
Przewidywana wartość: 590.3770141601562
Rzeczywista wartość: 569.8499755859375
Przykład 3:
Przewidywana wartość: 554.3075561523438
Rzeczywista wartość: 450.54998779296875
Przykład 4:
Przewidywana wartość: 685.8442993164062
Rzeczywista wartość: 839.4600219726562
Przykład 5:
Przewidywana wartość: 629.7590942382812
Rzeczywista wartość: 705.010009765625
Przykład 6:
Przewidywana wartość: 624.9415283203125
Rzeczywista wartość: 697.9400024414062
Przykład 7:
Przewidywana wartość: 682.2539672851562
Rzeczywista wartość: 854.9099731445312
Przykład 8:
Przewidywana

# GCNConv

In [63]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
    

class GraphRegressionModel(nn.Module):
    def __init__(self, num_node_features=1, hidden_dim=64, output_dim=1, dropout=0.5):
        super(GraphRegressionModel, self).__init__()

        # Graph Convolutional Layer
        self.conv1 = GCNConv(num_node_features, hidden_dim)


        # Fully Connected Layers with Dropout
        self.fc1 = nn.Linear(hidden_dim + 2, hidden_dim)
        self.dropout = nn.Dropout(p=dropout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index, edge_weight, global_feature = data.x, data.edge_index, data.edge_weight, data.global_feature

        # Apply Graph Convolution
        x = self.conv1(x, edge_index, edge_weight=edge_weight)


        # Global features concatenation
        global_feature = global_feature.expand(x.size(0), -1)
        x = torch.cat([x, global_feature], dim=1)

        # Fully Connected Layers with Dropout
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x

    def loss(self, pred, score):
        return F.mse_loss(pred, score)

# Zbieranie danych

In [64]:
from hyperopt import fmin, tpe, hp

# Zdefiniuj funkcję oceny (score function)
def objective(params):
    hidden_dim = int(params['hidden_dim'])
    dropout = params['dropout']

    model = GraphRegressionModel(hidden_dim=hidden_dim, dropout=dropout).to(device)
    # Definiuj optymalizator, liczbę epok, itp.
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    num_epochs = 200
    
    for epoch in range(num_epochs):
        # Trening i ewaluacja modelu
        for data in train_loader:
            data.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = model.loss(output, data.y.view(-1, 1).float())
            loss.backward()
            optimizer.step()
    
    # Ewaluacja na zbiorze testowym
    test_loss = 0.0  
    for data in test_loader:
        data.to(device)
        output = model(data)
        test_loss += model.loss(output, data.y.view(-1, 1).float()).item()
    
    # Zwróć funkcję oceny (score)
    return test_loss

# Przestrzeń poszukiwań dla hyperopt
space = {
    'hidden_dim': hp.quniform('hidden_dim', 32, 256, 32),  # Przeszukuj wartości co 32
    'dropout': hp.uniform('dropout', 0.05, 0.8),
}

# Minimalizacja funkcji oceny za pomocą algorytmu TPE
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10)  # Dla przykładu ustawiono max_evals na 10
print("Najlepsze hiperparametry:", best)
# Open the file in 'a' mode (append mode)

import torch

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Ustawienie optymalnych hiperparametrów
optimal_hidden_dim = int(best['hidden_dim'])
optimal_dropout = best['dropout']

#optimal_hidden_dim = 192
#optimal_dropout = 0.06

# Inicjalizacja modelu z optymalnymi hiperparametrami
optimal_model = GraphRegressionModel(hidden_dim=optimal_hidden_dim, dropout=optimal_dropout).to(device)



# Definiuj optymalizator, liczbę epok, itp.
optimizer = torch.optim.Adam(optimal_model.parameters(), lr=0.001)
num_epochs = 100

# Listy do śledzenia train loss i test loss
train_losses = []
test_losses = []

# Trening modelu
for epoch in range(num_epochs):
    if epoch % 10 == 0:
        print("Epoch:", epoch)
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = optimal_model(data)
        loss = optimal_model.loss(output, data.y.view(-1, 1).float())
        loss.backward()
        optimizer.step()

sample_data = test_dataset[0].to(device)

# Przeprowadzenie predykcji na przykładowym obiekcie
optimal_model.eval()
with torch.no_grad():
    prediction = optimal_model(sample_data)[0][0].item()

# Wyświetlenie wyników
print("Przewidywana wartość:", prediction)
print("Rzeczywista wartość:", sample_data.y.item())

total_accuracy = 0.0

# Iteracja przez cały zbiór testowy
for i, sample_data in enumerate(test_dataset):
    # Przeprowadzenie predykcji na przykładowym obiekcie
    optimal_model.eval()
    with torch.no_grad():
        prediction = optimal_model(sample_data.to(device))[0][0].item()
    
    # Wyświetlenie wyników
    print(f"Przykład {i + 1}:")
    print("Przewidywana wartość:", prediction)
    print("Rzeczywista wartość:", sample_data.y.item())
    
    # Obliczenie procentowej jakości regresji
    actual_value = sample_data.y.item()
    accuracy = 100 * (1 - abs(prediction - actual_value) / actual_value)
    total_accuracy += accuracy
    
    # print(f"Procentowa jakość regresji: {accuracy:.2f}%\n")

# Obliczenie średniej jakości procentowej
average_accuracy = total_accuracy / len(test_dataset)
print(f"Średnia procentowa jakość regresji dla wszystkich przykładów: {average_accuracy:.2f}%")

from sklearn.metrics import r2_score, mean_squared_error

# Załóżmy, że y_true i y_pred to twoje rzeczywiste i przewidywane wartości.
#optimal_model.load_state_dict(torch.load("model_big"))

total_accuracy = 0.0
y_true = []
y_pred = []

# Iteracja przez cały zbiór testowy
for i, sample_data in enumerate(test_dataset):
    # Przeprowadzenie predykcji na przykładowym obiekcie
    optimal_model.eval()
    with torch.no_grad():
        prediction = optimal_model(sample_data)[0][0].item()
        y_pred.append(prediction)
    
    y_true.append(sample_data.y.item())
    
    # Wyświetlenie wyników
    # print(f"Przykład {i + 1}:")
    # print("Przewidywana wartość:", prediction)
    # print("Rzeczywista wartość:", sample_data.y.item())
    
    # Obliczenie procentowej jakości regresji
    actual_value = sample_data.y.item()
    accuracy = 100 * (1 - abs(prediction - actual_value) / actual_value)
    total_accuracy += accuracy
    
    # print(f"Procentowa jakość regresji: {accuracy:.2f}%\n")

# Obliczenie średniej jakości procentowej
average_accuracy = total_accuracy / len(test_dataset)
r2 = r2_score(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
rmse = mse**(1/2)

print(f"R2 Score = {r2:.4f}")
print(f"MSE Score = {mse:.4f}")
print(f"RMSE Score = {rmse:.4f}")

y_mean = sum(y_true) / len(y_true)
print(f"Jaki procent zmiennej zależnej stanowi błąd RMSE? {rmse / y_mean * 100:.4f}%")

with open("file.txt", "a") as file:
    # Append the text to the file
    file.write("\n" + "GCNConv")  # Add a new line before appending
    file.write("\n" + "R2 "+ str(r2) )
    file.write("\n" + "MSE "+str( mse))
    file.write("\n" + "RMSE "+ str(rmse))
    file.write("\n" + "Awerage "+ str(average_accuracy))

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/tmp/ipykernel_4183/1307265366.py:39: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([28, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(pred, score)



100%|██████████| 10/10 [11:06<00:00, 66.68s/trial, best loss: 83090.6840209961] 
Najlepsze hiperparametry: {'dropout': 0.15435357993747928, 'hidden_dim': 160.0}
Epoch: 0
Epoch: 10
Epoch: 20
Epoch: 30
Epoch: 40
Epoch: 50
Epoch: 60
Epoch: 70
Epoch: 80
Epoch: 90
Przewidywana wartość: 749.0038452148438
Rzeczywista wartość: 874.5499877929688
Przykład 1:
Przewidywana wartość: 749.0038452148438
Rzeczywista wartość: 874.5499877929688
Przykład 2:
Przewidywana wartość: 609.9280395507812
Rzeczywista wartość: 569.8499755859375
Przykład 3:
Przewidywana wartość: 547.6429443359375
Rzeczywista wartość: 450.54998779296875
Przykład 4:
Przewidywana wartość: 774.7850341796875
Rzeczywista wartość: 839.4600219726562
Przykład 5:
Przewidywana wartość: 677.9342041015625
Rzeczywista wartość: 705.010009765625
Przykład 6:
Przewidywana wartość: 669.6142578125
Rzeczywista wartość: 697.9400024414062
Przykład 7:
Przewidywana wartość: 768.583740234375
Rzeczywista wartość: 854.9099731445312
Przykład 8:
Przewidywana war

# Ocena jakości

# Skrypt do wykonania